# API Basics — Rubric & Async/Await Walkthrough

Objective: Capture the rubric criteria from the provided GitHub issue and show concrete examples of using the existing `Requestor` (api.js) and `Handler` (handler.js) to demonstrate `async`/`await`, parameterized API calls (checkbox -> `genreId`), and rendering.


## Rubric (summary)

The rubric scores projects on understanding of APIs and best practices.

- Demonstrated Understanding (40 pts): Explain API concepts, `async`/`await`, JSON format, and how results are displayed (15 + 15 + 10).
- Code Architecture & Modularity (20 pts): Separate API layer, models, and rendering; keep code reusable and extensible (20 pts).
- Correctness of API Usage & Async Handling (20 pts): Clean, parameterized requests, safe JSON parsing, correct `async`/`await` usage.
- Error Handling & Robustness (10 pts): Validate inputs, handle empty results and network errors gracefully.
- Code Quality & Style (10 pts): Readable, idiomatic JS with comments and documentation.


## How this notebook maps to the rubric

- Show the `Requestor` as the API layer (separation of concerns).
- Show `Handler` as the display layer (rendering responsibility).
- Demonstrate `async`/`await` usage with error handling and JSON parsing.
- Show how adding a checkbox (genre filter) is a parameterized request that tests correctness and modularity.


## Requestor (api.js) — key points

`Requestor` is a small API wrapper that builds query strings and performs `fetch()` calls. Important parts to highlight: `buildQueryString`, `search`, and `request` which does the `fetch` and `await response.json()` parsing.


```javascript
// Example: using Requestor with async/await
import { Requestor } from '../assets/js/itunes/api.js';
const API_URL = 'https://itunes.apple.com';
const requestor = new Requestor(API_URL);

async function exampleSearch() {
  try {
    // await is used to pause until the promise from fetch resolves
    const data = await requestor.search({ term: 'adele', limit: 5, media: 'music' });
    console.log('Got results:', data);
    // Hand off to a display layer (Handler) or renderer here
  } catch (err) {
    // Proper error handling satisfies the rubric's robustness criteria
    console.error('Search failed', err);
  }
}

// Call the function (in a browser environment)
// exampleSearch();
```


## Handler (handler.js) — rendering results

`Handler` is responsible for turning API results into DOM elements. It exposes `clearResults()`, `displayResults(data)`, `handleResponse(data)`, and `handleError(err)`. This separation keeps rendering logic out of the API layer (good architecture).


```javascript
// Example: passing Requestor results to Handler (async/await + display)
import { Requestor } from '../assets/js/itunes/api.js';
import { Handler } from '../assets/js/itunes/handler.js';

const requestor = new Requestor('https://itunes.apple.com');
const handler = new Handler('result'); // id of the <tbody> container

async function searchAndShow(term) {
  try {
    const data = await requestor.search({ term, media: 'music', limit: 10 });
    // handler.handleResponse delegates to displayResults internally
    handler.handleResponse(data);
  } catch (err) {
    handler.handleError(err);
  }
}

// searchAndShow('coldplay');
```


## Checkbox (genre) example — how a UI change highlights rubric items

Adding a small checkbox for `Pop?` (genreId = 14) shows:

- Parameterized requests (Correctness of API Usage) — the API call changes based on UI input, demonstrating robust, testable behavior.
- Modularity — the UI reads the checkbox and builds an options object passed to `Requestor` (keeps concerns separated).
- Error-handling/testing — you can test that the `genreId` is present in the built query string and that results match expected genres.


```javascript
// Checkbox integration snippet (from music-api.md)
function buildSearchOptionsFromUI() {
  const term = document.getElementById('filterInput').value.trim();
  const opts = { term, limit: 20, media: 'music' };
  const popEl = document.getElementById('genrePop');
  if (popEl && popEl.checked) opts.genreId = 14;
  return opts;
}

async function searchFromUI() {
  const opts = buildSearchOptionsFromUI();
  try {
    const data = await requestor.search(opts);
    handler.handleResponse(data);
  } catch (err) {
    handler.handleError(err);
  }
}
```


## How this demonstrates rubric points (quick mapping)

- `async/await` usage: `await requestor.search(opts)` — non-blocking, easy-to-read flow (Correctness of Async Handling).
- API layering: `Requestor` builds the URL and handles fetch; `Handler` only deals with DOM (Architecture & Modularity).
- Parameterized requests: Checkbox -> `genreId` shows extensibility & correctness.
- Error handling: `try/catch` around `await` and using `handler.handleError` for UI-visible feedback (Robustness).


## Quick walkthrough — what to test manually:

1. Open the page that uses `music-api.md` (or serve the repo).
2. Type a search term and click `Search`. Verify console logs from `runTests()` and that results appear.
3. Check the `Pop?` checkbox and search again. Confirm the query string includes `genreId=14` in the Network tab.
4. Simulate network failure (offline) and confirm `handler.handleError` shows a user-friendly message.
5. Inspect code separation: `Requestor` should only build URLs and fetch; `Handler` should only create DOM nodes.


## References & links

- `assets/js/itunes/api.js` (Requestor) — builds queries and performs fetch().
- `assets/js/itunes/handler.js` (Handler) — display logic for results.
- `hacks/music-api.md` — page that wires Requestor + Handler and includes the `Pop?` checkbox example.

